In [1]:
import pandas as pd

try:
    df = pd.read_csv('../data/feed-views.log',
                    sep="\t",
                    names=["datetime", "user"])
except FileNotFoundError as e:
    print("File not found.")
    raise e

#  Конвертируем datetime в datetime64[ns] тип
df['datetime'] = pd.to_datetime(df['datetime'])

# Извлекаем компоненты времени: year, month, day, hour, minute, second
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour
df['minute'] = df['datetime'].dt.minute
df['second'] = df['datetime'].dt.second

#  Создаем колонку daytime используя метод cut
# Интервалы: 0-3:59 night, 4-6:59 early morning, 7-10:59 morning,
#           11-16:59 afternoon, 17-19:59 early evening, 20-23:59 evening
bins = [0, 4, 7, 11, 17, 20, 24]
labels = ['night', 'early morning', 'morning', 'afternoon', 'early evening', 'evening']

df['daytime'] = pd.cut(df['hour'],
                         bins=bins,
                         labels=labels,
                         right=False,  # левая граница включена [0,4), [4,7), [7,11) и т.д.
                         include_lowest=True)

# Устанавливаем колонку user как индекс
df.set_index('user', inplace=True)


In [2]:
  # Вычисляем количество элементов
print(f"Общее количество элементов: \n{df.count()}")

Общее количество элементов: 
datetime    1076
year        1076
month       1076
day         1076
hour        1076
minute      1076
second      1076
daytime     1076
dtype: int64


In [3]:
# Количество элементов в каждой категории времени дня используя value_counts()
print(f"Распределение по времени дня:\n{df['daytime'].value_counts()}")

Распределение по времени дня:
daytime
evening          509
afternoon        252
early evening    145
night            129
morning           36
early morning      5
Name: count, dtype: int64


In [4]:

# Сортируем по hour, minute, second одновременно (ascending) - модифицируем views напрямую
df.sort_values(['hour', 'minute', 'second'], inplace=True)

# Вычисляем минимум и максимум для часов, моду для daytime категорий
print(f"\nСтатистика:")
print(f"Минимальный час: {df['hour'].min()}")
print(f"Максимальный час: {df['hour'].max()}")
print(f"Мода для daytime: {df['daytime'].mode()[0]}")


Статистика:
Минимальный час: 0
Максимальный час: 23
Мода для daytime: evening


In [5]:
# Максимум часов для строк где время дня = 'night'
night_data = df[df['daytime'] == 'night']
if not night_data.empty:
    print(f"Максимальный час для night: 0{night_data['hour'].max()}:00")

Максимальный час для night: 03:00


In [6]:
# Минимум часов для строк где время дня = 'morning' + кто посещал
morning_data = df[df['daytime'] == 'morning']
if not morning_data.empty:
    morning_min_hour = morning_data['hour'].min()
    print(f"Минимальный час для morning: 0{morning_min_hour}:00")
    print(f"Пользователи посетители в {morning_min_hour}:00: {' '.join(morning_data[morning_data['hour'] == morning_min_hour].index.tolist())}")

Минимальный час для morning: 08:00
Пользователи посетители в 8:00: alexander alexander


In [7]:
# Вычисляем моду(самок частое значение) для hour и daytime
print(f"\nМода для hour: {' '.join(map(str, df['hour'].mode().tolist()))}")
print(f"Мода для daytime: {' '.join(df['daytime'].mode().tolist())}")


Мода для hour: 22
Мода для daytime: evening


In [8]:
# 3 самых ранних и поздних часа утром используя nsmallest() и nlargest()
if not morning_data.empty:
    print("\n3 самых ранних часа утром:")
    print(morning_data.nsmallest(3, 'hour')[['hour', 'datetime']])
    print("\n3 самых поздних часа утром:")
    print(morning_data.nlargest(3, 'hour')[['hour', 'datetime']])


3 самых ранних часа утром:
           hour                   datetime
user                                      
alexander     8 2020-05-15 08:16:03.918402
alexander     8 2020-05-15 08:35:01.471463
alexander     9 2020-05-15 09:02:24.999438

3 самых поздних часа утром:
            hour                   datetime
user                                       
konstantin    10 2020-05-16 10:02:08.925840
maxim         10 2020-05-14 10:08:04.832361
maxim         10 2020-05-14 10:08:05.726956


In [9]:
# используем describe() для получения базовой статистики для колонок
hour_stats = df['hour'].describe()
print(f"\nБазовая статистика для часов (describe):")
print(hour_stats)


Базовая статистика для часов (describe):
count    1076.000000
mean       16.249071
std         6.955490
min         0.000000
25%        13.000000
50%        19.000000
75%        22.000000
max        23.000000
Name: hour, dtype: float64


In [10]:

# Находим самый популярный интервал посещений - вычисляем IQR для hour
# Извлекаем значения из результата describe() и сохраняем в переменную iqr
iqr = hour_stats['75%'] - hour_stats['25%']

In [11]:

print(f"\nИнтерквартильный размах (IQR) для часов: {iqr}")
print(f"Самый популярный интервал посещений: {hour_stats['25%']} - {hour_stats['75%']} часов")



Интерквартильный размах (IQR) для часов: 9.0
Самый популярный интервал посещений: 13.0 - 22.0 часов
